# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [6]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [7]:
!cat talks.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [8]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,url_slug,speakers,date,title,venue,type,location,talk_url,description
0,JH13,"Hawes, JK.",2024-03-05,Comparing the carbon footprints of urban and c...,"Invited Seminar, Department of Sustainable Agr...",Online Oral.,"University of Patras, Patras, Greece.",NaN,NaN
1,JH12,"Hawes, JK, Gounaridis, D., Goldstein, B., Newe...",2023-11-10,"Urban Agriculture: Good for People, Places, Pl...",Center for Sustainable Systems External Adviso...,Oral.,"University of Michigan, Ann Arbor, MI, USA.",NaN,NaN
2,JH11,"Hawes, JK.",2023-10-05,Invited speaker: Climate Change and the Univer...,American Studies 301.,Online Oral.,"Purdue University, West Lafayette, IN, USA.",NaN,NaN
3,JH10,"Hawes, JK.",2023-02-14,Panel moderator for Climate Change: Part of th...,Indiana Council on World Affairs.,Online Moderator.,"Indianapolis, IN, USA.",NaN,NaN
4,JH9,"Hawes, JK.",2022-12-01,Invited guest lecture: Decolonizing Sustainabi...,"Environ 207, Sustainability and Society.",Oral.,"University of Michigan, Ann Arbor, MI, USA.",NaN,NaN
5,JH8,"Hawes, JK.",2022-11-30,Invited panelist: Closing the Loop: Opportunit...,Industrial Ecology Day.,Online Oral.,"University of California-Davis, Davis, CA, USA.",https://www.youtube.com/watch?v=zUlwaabElS0,NaN
6,JH7,"Hawes, JK, Gounaridis, D., Goldstein, B., Newe...",2022-10-01,Invited guest lecture: Urban Agriculture: Good...,"SEAS 517, Urban Sustainability.",Oral.,"University of Michigan, Ann Arbor, MI, USA.",NaN,NaN
7,JH6,"Hawes, JK.",2022-02-01.,Invited panel moderator: Water Systems.,Michigan University-Wide Sustainability & Envi...,Online Moderator.,"University of Michigan, Ann Arbor, MI, USA.",NaN,NaN
8,JH5,"Hawes, JK.",2020-07-30.,Climate Change: Part of the Great Decisions sp...,Indiana Council on World Affairs.,Online Oral.,"Indianapolis, IN, USA.",NaN,NaN
9,JH4,"Schirm, V, Hawes, JK, Russel, M.",2018-09-01.,Invited guest lecture: Deep Leadership and App...,"EEE 290, Professional Development Seminar.",Oral.,"Purdue University, West Lafayette, IN, USA.",NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [9]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [10]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n" # This is where we'd add different collections for formatting. 
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [11]:
!ls ../_talks

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
!cat ../_talks/2013-03-01-tutorial-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
